# ***Analisis_Sentimiento:***

### Importar librerias

In [40]:
import pandas as pd
import pyarrow as py
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from textblob import Word
import pyarrow.parquet as pq

In [41]:
# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True



---

### Cargar el DataFrame df_user_review

In [42]:
# Se genera una variable con el archivo parquet
df_user_reviews = pd.read_csv('C:/Users/flore/OneDrive/Desktop/GRUPAL HENRY/google/analisisentimiento/pensalvvaniasent.csv')


C:\Users\flore\AppData\Local\Temp\ipykernel_1580\4010840695.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user_reviews = pd.read_csv('C:/Users/flore/OneDrive/Desktop/GRUPAL HENRY/google/analisisentimiento/pensalvvaniasent.csv')


In [43]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [44]:
# Visualizacion del DataFrame
df_user_reviews.head(10)

,name_x,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,Service options,Amenities,Atmosphere,Crowd,Highlights,Popular for,Offerings,Dining options,Planning,Health & safety,Accessibility,Payments,From the business,Health and safety,user_id,name_y,time_usuario,rating,text_usuario,time,text,year
0,Crown Fried Chicken,"Crown Fried Chicken, 208 East St, Bloomsburg, ...",0x89c587762e7e69dd:0x30a60dce48b073c3,41.004147,-76.453729,restaurant,4.2,18,$,"Thursday 11AM–3AM, Friday 11AM–3AM, Saturday 1...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,Solo dining,Quick bite,NaN,Accepts reservations,NaN,NaN,NaN,NaN,NaN,116240853927540430813,Hamid Mukhtar,2021-08-01 21:49:20.963,5,Inexpensive and delicious food. Reminds me of ...,NaN,NaN,2021
1,Crown Fried Chicken,"Crown Fried Chicken, 208 East St, Bloomsburg, ...",0x89c587762e7e69dd:0x30a60dce48b073c3,41.004147,-76.453729,restaurant,4.2,18,$,"Thursday 11AM–3AM, Friday 11AM–3AM, Saturday 1...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,Solo dining,Quick bite,NaN,Accepts reservations,NaN,NaN,NaN,NaN,NaN,108987040068572830646,Alex King,2021-08-09 16:22:47.231,3,pros\nstaff were nice\nprices were good for qu...,NaN,NaN,2021
2,Crown Fried Chicken,"Crown Fried Chicken, 208 East St, Bloomsburg, ...",0x89c587762e7e69dd:0x30a60dce48b073c3,41.004147,-76.453729,restaurant,4.2,18,$,"Thursday 11AM–3AM, Friday 11AM–3AM, Saturday 1...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,Solo dining,Quick bite,NaN,Accepts reservations,NaN,NaN,NaN,NaN,NaN,109218205375471507404,Jonathan Reed,2021-07-23 17:20:42.318,2,"The chicken was nice and crispy, but it had a ...",2021-07-25 04:08:11.632999936,Jonathan thanks for you review please stop by ...,2021
3,Crown Fried Chicken,"Crown Fried Chicken, 208 East St, Bloomsburg, ...",0x89c587762e7e69dd:0x30a60dce48b073c3,41.004147,-76.453729,restaurant,4.2,18,$,"Thursday 11AM–3AM, Friday 11AM–3AM, Saturday 1...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,Solo dining,Quick bite,NaN,Accepts reservations,NaN,NaN,NaN,NaN,NaN,103615682383180087623,Ben Johnson,2021-07-15 01:11:56.491,5,Loved the lamb over rice! Chicken is good but...,2021-07-25 04:11:03.324000000,Thanks Ben really appreciate your review 🙏 tha...,2021
4,Crown Fried Chicken,"Crown Fried Chicken, 208 East St, Bloomsburg, ...",0x89c587762e7e69dd:0x30a60dce48b073c3,41.004147,-76.453729,restaurant,4.2,18,$,"Thursday 11AM–3AM, Friday 11AM–3AM, Saturday 1...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,Solo dining,Quick bite,NaN,Accepts reservations,NaN,NaN,NaN,NaN,NaN,114033787332228485618,Chantel Blink,2021-07-22 09:37:20.849,3,Good for last min dinner,NaN,NaN,2021
5,Judges Corner,"Judges Corner, 848 E 5th St, Erie, PA 16507",0x882d7fc7b02c7a39:0xedc083a71e5be96d,42.137736,-80.066740,soul food restaurant,4.2,8,NaN,"Thursday Closed, Friday 12–8PM, Saturday 12–8P...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,"Lunch, Dinner, Solo dining","Comfort food, Quick bite",Dessert,NaN,NaN,NaN,NaN,NaN,NaN,111342712545574369111,Jamesha White,2021-07-19 03:24:13.854,4,The smothered pork chops are banging!! Yams ar...,NaN,NaN,2021
6,Judges Corner,"Judges Corner, 848 E 5th St, Erie, PA 16507",0x882d7fc7b02c7a39:0xedc083a71e5be96d,42.137736,-80.066740,soul food restaurant,4.2,8,NaN,"Thursday Closed, Friday 12–8PM, Saturday 12–8P...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,"Lunch, Dinner, Solo dining","Comfort food, Quick bite",Dessert,NaN,NaN,NaN,NaN,NaN,NaN,109279853426078527502,Larry Barnes (Sundance),2021-05-11 17:46:39.531,4,I was with someone else that went there they s...,NaN,NaN,2021
7,Judges Corner,"Judges Corner, 848 E 5th St, Erie, PA 16507",0x882d7fc7b02c7a39:0xedc083a71e5be96d,42.137736,-80.066740,soul food restaurant,4.2,8,NaN,"Thursday Closed, Friday 12–8PM, Saturday 12–8P...","Takeout, Dine-in, Delivery",Good for kids,Casual,NaN,NaN,"Lunch, Dinner, Solo dining","Comfort food, Quick bite",

In [45]:
df_user_reviews.columns

Index(['name_x', 'address', 'gmap_id', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'price', 'hours', 'Service options',
       'Amenities', 'Atmosphere', 'Crowd', 'Highlights', 'Popular for',
       'Offerings', 'Dining options', 'Planning', 'Health & safety',
       'Accessibility', 'Payments', 'From the business', 'Health and safety',
       'user_id', 'name_y', 'time_usuario', 'rating', 'text_usuario', 'time',
       'text', 'year'],
      dtype='object')

In [46]:
df_user_reviews = df_user_reviews[df_user_reviews['name_x'].str.contains(r'\bsubway\b', case=False)]

In [47]:
df_user_reviews = df_user_reviews[df_user_reviews["name_x"] == "Subway"]

In [48]:
df_user_reviews.head(5)

,name_x,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,Service options,Amenities,Atmosphere,Crowd,Highlights,Popular for,Offerings,Dining options,Planning,Health & safety,Accessibility,Payments,From the business,Health and safety,user_id,name_y,time_usuario,rating,text_usuario,time,text,year
297,Subway,"Subway, 5360 Lincoln Hwy Suite #1, Gap, PA 17527",0x89c646f3f822e73d:0x19bfe62785468634,39.9898,-76.017344,"sandwich shop, caterer, fast food restaurant, ...",3.2,58,$,"Tuesday 9AM–8PM, Wednesday 9AM–8PM, Thursday 9...","Delivery, Takeout, Dine-in",Good for kids,Casual,Tourists,Fast service,"Lunch, Dinner, Solo dining","Healthy options, Kids' menu, Quick bite, Veget...","Lunch, Catering",NaN,NaN,"Wheelchair accessible entrance, Wheelchair acc...","Debit cards, NFC mobile payments",NaN,NaN,108922831758235567259,ryanrhea1,2020-10-21 14:25:37.389,1,I went there only to ask for fork and spoon bu...,NaN,NaN,2020
298,Subway,"Subway, 5360 Lincoln Hwy Suite #1, Gap, PA 17527",0x89c646f3f822e73d:0x19bfe62785468634,39.9898,-76.017344,"sandwich shop, caterer, fast food restaurant, ...",3.2,58,$,"Tuesday 9AM–8PM, Wednesday 9AM–8PM, Thursday 9...","Delivery, Takeout, Dine-in",Good for kids,Casual,Tourists,Fast service,"Lunch, Dinner, Solo dining","Healthy options, Kids' menu, Quick bite, Veget...","Lunch, Catering",NaN,NaN,"Wheelchair accessible entrance, Wheelchair acc...","Debit cards, NFC mobile payments",NaN,NaN,106115248228316282405,Mable Walton,2017-10-08 22:31:56.424,5,I ordered a veggie sub and it was one of the b...,NaN,NaN,2017
299,Subway,"Subway, 5360 Lincoln Hwy Suite #1, Gap, PA 17527",0x89c646f3f822e73d:0x19bfe62785468634,39.9898,-76.017344,"sandwich shop, caterer, fast food restaurant, ...",3.2,58,$,"Tuesday 9AM–8PM, Wednesday 9AM–8PM, Thursday 9...","Delivery, Takeout, Dine-in",Good for kids,Casual,Tourists,Fast service,"Lunch, Dinner, Solo dining","Healthy options, Kids' menu, Quick bite, Veget...","Lunch, Catering",NaN,NaN,"Wheelchair accessible entrance, Wheelchair acc...","Debit cards, NFC mobile payments",NaN,NaN,109198090834891395547,Curtis Lahr,2019-06-24 14:04:36.984,2,Good food and friendly service. Seems like the...,NaN,NaN,2019
300,Subway,"Subway, 5360 Lincoln Hwy Suite #1, Gap, PA 17527",0x89c646f3f822e73d:0x19bfe62785468634,39.9898,-76.017344,"sandwich shop, caterer, fast food restaurant, ...",3.2,58,$,"Tuesday 9AM–8PM, Wednesday 9AM–8PM, Thursday 9...","Delivery, Takeout, Dine-in",Good for kids,Casual,Tourists,Fast service,"Lunch, Dinner, Solo dining","Healthy options, Kids' menu, Quick bite, Veget...","Lunch, Catering",NaN,NaN,"Wheelchair accessible entrance, Wheelchair acc...","Debit cards, NFC mobile payments",NaN,NaN,117938450544763974299,Drew Martin,2019-04-14 11:24:38.977,4,"Good, fast service however staff seemed a tad ...",NaN,NaN,2019
301,Subway,"Subway, 5360 Lincoln Hwy Suite #1, Gap, PA 17527",0x89c646f3f822e73d:0x19bfe62785468634,39.9898,-76.017344,"sandwich shop, caterer, fast food restaurant, ...",3.2,58,$,"Tuesday 9AM–8PM, Wednesday 9AM–8PM, Thursday 9...","Delivery, Takeout, Dine-in",Good for kids,Casual,Tourists,Fast service,"Lunch, Dinner, Solo dining","Healthy options, Kids' menu, Quick bite, Veget...","Lunch, Catering",NaN,NaN,"Wheelchair accessible entrance, Wheelchair acc...","Debit cards, NFC mobile payments",NaN,NaN,106144401855209333184,J,2019-09-08 22:30:33.375,2,I use to like this subway but today was not it...,NaN,NaN,2019


In [53]:
sentimiento =df_user_reviews[['name_x', 'gmap_id', 'avg_rating', 'num_of_reviews', 'Service options', 'user_id' , 'rating',
'text_usuario','time_usuario']]

In [54]:
sentimiento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11176 entries, 297 to 404604
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name_x           11176 non-null  object 
 1   gmap_id          11176 non-null  object 
 2   avg_rating       11176 non-null  float64
 3   num_of_reviews   11176 non-null  int64  
 4   Service options  11176 non-null  object 
 5   user_id          11176 non-null  object 
 6   rating           11176 non-null  int64  
 7   text_usuario     5391 non-null   object 
 8   time_usuario     11176 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 873.1+ KB



### Crea funcion para limpiar el texto


In [7]:
# Función para limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales y puntuación
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])  # Eliminar palabras de parada
    text = ' '.join([Word(word).lemmatize() for word in text.split()])  # Lematización

    return text



---



---


### Crea funcion para Analisis de Sentimiento


In [55]:
# Definir una función para el análisis de sentimiento
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity < 0:
        return 0  # Malo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 2  # Positivo

In [52]:
df_user_reviews['text_usuario'].fillna('sin datos')

297       I went there only to ask for fork and spoon bu...
298       I ordered a veggie sub and it was one of the b...
299       Good food and friendly service. Seems like the...
300       Good, fast service however staff seemed a tad ...
301       I use to like this subway but today was not it...
                                ...                        
404600                                            sin datos
404601                                            sin datos
404602                                            sin datos
404603                                            sin datos
404604                                            sin datos
Name: text_usuario, Length: 11176, dtype: object

In [18]:
# Aplicar limpieza a la columna 'Review'
df_user_reviews['clean_reviews'] = df_user_reviews['text_usuario'].apply(clean_text)

AttributeError: 'float' object has no attribute 'lower'

In [56]:
import pandas as pd
import nltk
import re



def clean_text(text):
    # Ensure text is a string
    text = str(text)

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters and punctuation
    tokens = text.split()  # Split into words

    # Download NLTK stopwords and lemmatizer if needed
    nltk.download('stopwords')
    lemmatizer = nltk.WordNetLemmatizer()

    # Remove stopwords (adjust list as needed)
    stopwords = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if token not in stopwords]

    # Lemmatize (adjust part-of-speech tag if needed)
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

sentimiento['clean_reviews'] = sentimiento['text_usuario'].apply(clean_text)

print(sentimiento)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

        name_x                                gmap_id  avg_rating  \
297     Subway  0x89c646f3f822e73d:0x19bfe62785468634         3.2   
298     Subway  0x89c646f3f822e73d:0x19bfe62785468634         3.2   
299     Subway  0x89c646f3f822e73d:0x19bfe62785468634         3.2   
300     Subway  0x89c646f3f822e73d:0x19bfe62785468634         3.2   
301     Subway  0x89c646f3f822e73d:0x19bfe62785468634         3.2   
...        ...                                    ...         ...   
404600  Subway  0x883261914c3daf3f:0x94130ce5345c8cdb         4.3   
404601  Subway  0x883261914c3daf3f:0x94130ce5345c8cdb         4.3   
404602  Subway  0x883261914c3daf3f:0x94130ce5345c8cdb         4.3   
404603  Subway  0x883261914c3daf3f:0x94130ce5345c8cdb         4.3   
404604  Subway  0x883261914c3daf3f:0x94130ce5345c8cdb         4.3   

        num_of_reviews                              Service options  \
297                 58                   Delivery, Takeout, Dine-in   
298                 58       

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

In [57]:
# Aplicar el análisis de sentimiento a la columna 'clean_reviews' y crear una nueva columna 'analisis_sentimientos'
sentimiento['analisis_sentimientos'] = sentimiento['clean_reviews'].apply(get_sentiment)

C:\Users\flore\AppData\Local\Temp\ipykernel_1580\233430876.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimiento['analisis_sentimientos'] = sentimiento['clean_reviews'].apply(get_sentiment)


In [58]:
# Visualizacion del dataFrame
sentimiento.head(10)

,name_x,gmap_id,avg_rating,num_of_reviews,Service options,user_id,rating,text_usuario,time_usuario,clean_reviews,analisis_sentimientos
297,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",108922831758235567259,1,I went there only to ask for fork and spoon bu...,2020-10-21 14:25:37.389,went ask fork spoon lady said allowed give spo...,2
298,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",106115248228316282405,5,I ordered a veggie sub and it was one of the b...,2017-10-08 22:31:56.424,ordered veggie sub one best ever everything wa...,2
299,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",109198090834891395547,2,Good food and friendly service. Seems like the...,2019-06-24 14:04:36.984,good food friendly service seems like staff re...,2
300,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",117938450544763974299,4,"Good, fast service however staff seemed a tad ...",2019-04-14 11:24:38.977,good fast service however staff seemed tad int...,2
301,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",106144401855209333184,2,I use to like this subway but today was not it...,2019-09-08 22:30:33.375,use like subway today sandwich mostly bread ba...,2
302,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",108841571319595781209,4,Always have great Subs,2019-03-31 02:43:47.349,always great sub,2
303,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",105008270303190784664,4,Good food just don't show up too late if you w...,2017-02-14 05:13:39.904,good food dont show late want meatball sub run...,2
304,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",103447206497067128123,1,Seriously the worst subway ever we had little ...,2018-04-27 22:45:12.866,seriously worst subway ever little meat sandwi...,0
305,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",115754598857543216009,5,NaN,2019-04-28 12:35:10.774,nan,1
306,Subway,0x89c646f3f822e73d:0x19bfe62785468634,3.2,58,"Delivery, Takeout, Dine-in",115254699056637248090,5,NaN,2021-04-06 00:27:46.063,nan,1


In [25]:
# Se elimina la columna 'Review'
df_user_reviews = df_user_reviews.drop(['Review'], axis=1)

### Guardar el DataFrame comprimido en Parquet


In [39]:
# Guardar el DataFrame comprimido en Parquet
sentimiento.to_csv('sentimientopa.csv')